In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from huggingface_hub import from_pretrained_keras

In [ ]:
backbone = from_pretrained_keras("keras-io/monocular-depth-estimation")
live_path = '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/live/504311.png'
spoof_path = '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/3613/spoof/541354.png'

In [ ]:
# Load the images
live_image = cv2.imread(live_path)
spoof_image = cv2.imread(spoof_path)

# Preprocess the images
live_image = cv2.cvtColor(live_image, cv2.COLOR_BGR2RGB)
spoof_image = cv2.cvtColor(spoof_image, cv2.COLOR_BGR2RGB)
live_image = cv2.resize(live_image, (224, 224))
spoof_image = cv2.resize(spoof_image, (224, 224))
live_image = tf.image.convert_image_dtype(live_image, tf.float32)
spoof_image = tf.image.convert_image_dtype(spoof_image, tf.float32)

live_image = tf.expand_dims(live_image, axis=0)
spoof_image = tf.expand_dims(spoof_image, axis=0)


# Make predictions
live_mask = backbone.predict(live_image)
spoof_mask = backbone.predict(spoof_image)

live_mask = live_mask[0]
spoof_mask = spoof_mask[0]

print(np.max(live_mask))
print(np.min(live_mask))
print(np.max(spoof_mask))
print(np.min(spoof_mask))

live_mask = (live_mask - np.min(live_mask)) / (np.max(live_mask) - np.min(live_mask)) * 255.0
spoof_mask = (spoof_mask - np.min(spoof_mask)) / (np.max(spoof_mask) - np.min(spoof_mask)) * 255.0

# Convert the masks to uint8
live_mask = live_mask.astype(np.uint8)
spoof_mask = spoof_mask.astype(np.uint8)

# Show the predicted masks
cv2.imshow("Live Mask", live_mask)
cv2.imshow("Spoof Mask", spoof_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()